# Getting Started - Lab 01 - Vectara Query API

We'll now explore the Vectara Query API.

This notebook will use our "lab" authentication profile, if you haven't set this up, please [Setup Authentication](./00_setup_authentication.ipynb).

In [ ]:
from vectara.factory import Factory
from vectara.managers import CreateCorpusRequest
import logging

logging.basicConfig(format='%(asctime)s:%(name)-35s %(levelname)s:%(message)s', level=logging.INFO, datefmt='%H:%M:%S %z')
logger = logging.getLogger(__name__)

client = Factory(profile="lab").build()


## Setup Corpus and Data
Before we can run queries, we setup a corpus and some data. We'll examine this in more depth in the following notebooks.

In [ ]:
request = CreateCorpusRequest(name="Getting Started - Query API", key="01-getting-started-query-api")
response = client.lab_helper.create_lab_corpus(request)

logger.info(f"Our corpus key is [{response.key}]")

In [ ]:
from pathlib import Path
from vectara.types import StructuredDocument

path = Path("resources/shakespeare/taming_shrew.txt")
with open(path, "r", encoding="utf-8") as f:
    play_text = f.read()
        
request = StructuredDocument.parse_obj({
   "id": "my-doc",
   "type": "structured",
   "title": "Taming of the Shrew",
   "description": "The Shakespeare play, 'the Taming of the Shrew'",
   "sections": [
       {
           "text": play_text # One big section which will be automatically chunked.
       }
   ]
})

client.documents.index(response.key, request=request)

# Query Types
Vectara supports three query interfaces, in addition to chat which will be covered seperately.

1. Simple Single Corpus Query
2. Advanced Single Corpus Query
3. Multiple Corpora Query 

We'll now examine how you can invoke each of these with the SDK.

## Single Corpus Query
The following code performs a query against a single corpus using the convenience method which invokes [Simple Single Corpus Query REST API](https://docs.vectara.com/docs/rest-api/search-corpus).

In [ ]:
import json

query = "What is the Wager in the play, 'The Taming of the Shrew'?"

query_response = client.queries.search(response.key, query="query")
logger.info(json.dumps(query_response.model_dump(), indent=4))

## Advanced Single Corpus Query
The following code performs a query against a single corpus using the advanced method which invokes [Advanced Single Corpus Query REST API](https://docs.vectara.com/docs/rest-api/query-corpus).

In [ ]:
from vectara.queries import SearchCorpusParameters
from vectara.types import GenerationParameters 

generation = GenerationParameters.parse_obj({
    
})

search_corpus = SearchCorpusParameters.parse_obj({
    # TODO Add reranker from SearchParameters#SearchReranker
    # TODO Add context_configuration from SearchParameters#ContextConfiguration
})

query_response = client.queries.query_corpus(response.key, query=query, search=search_corpus, generation=generation)
logger.info(query_response.summary)

# Multiple Corpora Query
We will now show how you can invoke the multiple-corpora query for advanced use cases which invoke [Multiple Corpora Query REST API](https://docs.vectara.com/docs/rest-api/query).

Please note the two similarly named types:
* `vectara.queries.SearchCorpusParameters` - Used above to search a single corpus
* `vectara.types.SearchCorporaParameters` - Used hear to search multiple corpora

Also observe the slight difference in methods in `vectara.queries.QueryClient`:
* `QueryClient#query_corpus` - Used above to search a single corpus
* `QueryClient#query` - Used here to perform a multi-corpora query

In [ ]:
from vectara.types import SearchCorporaParameters

search = SearchCorporaParameters.parse_obj({
    "corpora": [{"corpus_key": response.key}]
})


query_response = client.queries.query(query=query, search=search, generation=generation)
logger.info(query_response.summary)